<a href="https://colab.research.google.com/github/ndow33/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

#Engineer at least two new features. 

In [0]:
#Create total perks feature
df['total_perks'] = (df['elevator'] + df['cats_allowed'] + df['hardwood_floors'] +
                     df['dogs_allowed'] + df['doorman'] + df['dishwasher'] + 
                     df['no_fee'] + df['laundry_in_building'] + df['fitness_center'] +
                     df['pre-war'] + df['laundry_in_unit'] + df['roof_deck'] +
                     df['outdoor_space'] + df['dining_room'] + df['high_speed_internet'] +
                     df['balcony'] + df['swimming_pool'] + df['new_construction'] +
                     df['terrace'] + df['exclusive'] + df['loft'] + df['garden_patio'] +
                     df['wheelchair_access'] + df['common_outdoor_space'] + df['common_outdoor_space'])


In [22]:
#Create a bedroom to bathroom ratio feature
df['bed_to_bath'] = df['bedrooms']/df['bathrooms']
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_perks,bed_to_bath
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,2.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.0


In [41]:
#check for nan's
sum(np.isnan(df['bed_to_bath']))

151

#Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [24]:
df['created'].sort_values(ascending=True)

5186     2016-04-01 22:12:41
7945     2016-04-01 22:56:00
6424     2016-04-01 22:57:15
7719     2016-04-01 23:26:07
1723     2016-04-02 00:48:13
                ...         
19943    2016-06-29 17:47:34
16801    2016-06-29 17:56:12
32633    2016-06-29 18:14:48
20560    2016-06-29 18:30:41
17743    2016-06-29 21:41:47
Name: created, Length: 48817, dtype: object

In [25]:
#Split the data into train and test data
train = df[df['created'] < '2016-06-01']
train['created'].sort_values()

5186     2016-04-01 22:12:41
7945     2016-04-01 22:56:00
6424     2016-04-01 22:57:15
7719     2016-04-01 23:26:07
1723     2016-04-02 00:48:13
                ...         
15311    2016-05-31 22:07:36
20331    2016-05-31 22:39:35
35264    2016-05-31 22:46:46
22002    2016-05-31 22:46:47
15697    2016-05-31 23:10:48
Name: created, Length: 31844, dtype: object

In [26]:
test = df[df['created'] >= '2016-06-01']
test['created'].sort_values()

11474    2016-06-01 01:10:37
19176    2016-06-01 01:11:06
16226    2016-06-01 01:11:12
37756    2016-06-01 01:11:52
17946    2016-06-01 01:12:22
                ...         
19943    2016-06-29 17:47:34
16801    2016-06-29 17:56:12
32633    2016-06-29 18:14:48
20560    2016-06-29 18:30:41
17743    2016-06-29 21:41:47
Name: created, Length: 16973, dtype: object

#Fit a linear regression model with at least two features.

In [27]:
#Begin with 0 feature baseline
#What was the average price of rent in our training data?
train['price'].mean()

3575.604007034292

In [0]:
#What if this was our prediction for each rental price? How far off would we be?
#arrange y target vectors
target = 'price'
y_train = train[target]
y_test = test[target]

In [29]:
print('Mean baseline (using 0 features)')
guess = y_train.mean()
guess

Mean baseline (using 0 features)


3575.604007034292

In [30]:
#Train error
from sklearn.metrics import mean_absolute_error, r2_score
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train error (rental prices in April and May): ${mae:.2f}')

Train error (rental prices in April and May): $1201.88


In [31]:
#test error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test error (June rental prices): ${mae:.2f}')

Test error (June rental prices): $1197.71


In [0]:
#now that we have a baseline, we will use two features
#First, import the appropriate estimator class
from sklearn.linear_model import LinearRegression

In [0]:
#Second, instantiate the class
model = LinearRegression(fit_intercept=True)

In [34]:
#Third, Re-arrange X features matrices
features = ['total_perks', 'bedrooms']
print(f'Linear Regression, dependent on: {features}')

X_train = train[features]
X_test = test[features]

Linear Regression, dependent on: ['total_perks', 'bedrooms']


In [35]:
#Fourth, fit the model
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
print(f'Train error: ${train_mae:.2f}')

Train error: $924.81


In [36]:
#Fifth, apply the model to new data
y_pred_test = model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
print(f'Test Error: ${test_mae:.2f}')

Test Error: $934.50


#Get the model's coefficients and intercept.

In [37]:
print(f'The model intercept is ${model.intercept_:.2f}')
print(f'The model coefficients are ${model.coef_[0]:.2f} and ${model.coef_[1]:.2f}')
print(f'That means that according to the model, every additional perk an apartment has, the rent goes up by ${model.coef_[0]:.2f}')
print(f'And for each additional bedroom, the rent goes up by ${model.coef_[1]:.2f}')

The model intercept is $1753.90
The model coefficients are $122.65 and $811.66
That means that according to the model, every additional perk an apartment has, the rent goes up by $122.65
And for each additional bedroom, the rent goes up by $811.66


#Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.

In [38]:
from math import sqrt
print('Here are the metrics for the train data:')
print(f'Root mean square error(RMSE): ${sqrt(train_mae):.2f}')
print(f'Mean absolute error(MAE): ${train_mae:.2f}')
print(f'R squared(R^2): {r2_score(y_train, y_pred_train):.2f}')

Here are the metrics for the train data:
Root mean square error(RMSE): $30.41
Mean absolute error(MAE): $924.81
R squared(R^2): 0.35


In [39]:
print('Here are the metrics for the test data:')
print(f'Root mean square error(RMSE): ${sqrt(test_mae):.2f}')
print(f'Mean absolute error(MAE): ${test_mae:.2f}')
print(f'R squared(R^2): {r2_score(y_test, y_pred_test):.2f}')

Here are the metrics for the test data:
Root mean square error(RMSE): $30.57
Mean absolute error(MAE): $934.50
R squared(R^2): 0.35
